# Eat Safe, Love

## Notebook Set Up

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
db.list_collection_names()

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields= {'BusinessName':1, 'scores.Hygiene':1}

# Use count_documents to display the number of documents in the result
count= establishments.count_documents(query)

print("Number of documents in result:", count)
print()

# Display the first document in the results using pprint
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
results_df= pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print("The number of rows are:", len(results_df))

# Display the first 10 rows of the DataFrame
results_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': 
                                {'$regex':'London'},
                                                        'RatingValue': {'$gte': 4}}

field= {'BusinessName': 1, 'LocalAuthorityName':1, 'RatingValue':1}

results= establishments.find(query, field)

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of documents in result:", count)
print()

# Display the first document in the results using pprint
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
london_df= pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print("The number of rows are:", len(london_df))

# Display the first 10 rows of the DataFrame
london_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

query_one = {'BusinessName': 'Penang Flavours'}

fields = {'BusinessName': 1,'geocode.latitude': 1, 'geocode.longitude': 1}
result_one = establishments.find(query_one, fields)

degree_search = 0.01
latitude = result_one[0]['geocode']['latitude']
longitude = result_one[0]['geocode']['longitude']

# Find Top 5 
query = {'RatingValue': 5,
         'geocode.latitude': {'$gte': (latitude - degree_search)},
         'geocode.latitude': {'$lte': (latitude + degree_search)},
         'geocode.longitude': {'$gte': (longitude - degree_search)},
         'geocode.longitude': {'$lte': (longitude + degree_search)},
         'scores.Hygiene': {'$ne': None}
         }

show = {'BusinessName': 1,'RatingValue': 1, 'scores.Hygiene':1, 'geocode.latitude': 1, 'geocode.longitude': 1}

sort=[('scores.Hygiene', 1)]

limit=5

result = establishments.find(query, show).sort(sort).limit(limit)

# Print the results
# for i in range (5):
#     pprint(result[i])

BusinessName_list=[]
RatingValue_list=[]
id_list=[]
latitude_list=[]
longitude_list=[]
hygiene_list=[]

for row in result:
    pprint(row)
    BusinessName= row['BusinessName']
    RatingValue= row['RatingValue']
    id= row["_id"]
    latitude= row['geocode']['latitude']
    longitude= row['geocode']['longitude']
    hygiene = row ['scores']['Hygiene']
    
    BusinessName_list.append(BusinessName)
    RatingValue_list.append(RatingValue)
    id_list.append(id)
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    hygiene_list.append(hygiene)


In [ ]:
# Convert result to Pandas DataFrame
RatingValue_df = pd.DataFrame({
    
 'Business Name': BusinessName_list,
 "Rating Value" : RatingValue_list,
    "id": id_list,
    "latitude" : latitude_list,
    "longitude" : longitude_list,
    "Hygiene" : hygiene_list

})

RatingValue_df.head()

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
        {"$match": {"scores.Hygiene": 0}},
        {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
         {"$sort": {"count": -1}}
         
         ]

# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
result_df= pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Number of rows in a DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)